In [53]:
# !pip install dotenv
# !pip install paramiko
# !pip install fabric
# !pip install pytz

In [54]:
import os
from dotenv import load_dotenv
import subprocess
import re
import pytz
from datetime import datetime
from fabric import Connection
import requests
from pathlib import Path

load_dotenv()

True

In [64]:
# Load secrets from .env
IMMICH_API_KEY = os.getenv("IMMICH_API_KEY")
IMMICH_UPLOAD_URL = str(os.getenv("IMMICH_UPLOAD_URL"))
SERVER_ADDRESS = str(os.getenv("SERVER_ADDRESS"))
SERVER_USERNAME = str(os.getenv("SERVER_USERNAME"))
SERVER_PASSWORD = str(os.getenv("SERVER_PASSWORD"))

TMP_DIR = "./tmp"
REMOTE_ROOT = f"/zstorage/surveillance_video"
HIGH_QUALITY = "30"
LOW_QUALITY = "37" #Found through testing to be a good balance of size and not losing fidelity

DEVICE_ID = "living_room_rpi_cam"


In [65]:

# Create a connection using password authentication
conn = Connection(
    host=SERVER_ADDRESS,
    user=SERVER_USERNAME,
    connect_kwargs={
        "password": SERVER_PASSWORD
    }
)

# Verify it works
result = conn.run("hostname", hide=True)
print("Connected to:", result.stdout.strip())


Connected to: jserver


In [66]:
def convert_file(local_filepath):
    av1_filename = os.path.basename(local_filepath).split(".")[0] + ".mkv"
    av1_filepath = f"{TMP_DIR}/{av1_filename}"
    assert not os.path.exists(av1_filepath)

    cmd = [
        "ffmpeg",
        "-hwaccel", "cuda",
        "-i", local_filepath,
        "-c:v", "av1_nvenc",
        "-preset", 'p7',
        "-cq", LOW_QUALITY,
        "-c:a", "copy",
        av1_filepath
    ]

    subprocess.run(cmd, check=True)
    return av1_filepath

In [68]:
def send_image_to_immich(video_filepath):
    print("beginning to process the request")
    video_filename = Path(video_filepath).name
    print(video_filename)
    stats = os.stat(video_filepath)
    with open(video_filepath, 'rb') as f:
        headers = {'x-api-key': IMMICH_API_KEY}
        tz = pytz.timezone(os.getenv("TIMEZONE", "America/Denver"))
        files = {'assetData': f}
        file_modified_at_timestamp = datetime.fromtimestamp(stats.st_mtime, tz).isoformat()
        file_created_at_timestamp = datetime.fromtimestamp(stats.st_ctime, tz).isoformat()
        print(file_modified_at_timestamp)

        data = {
            'deviceId': DEVICE_ID,
            'deviceAssetId': video_filename,
            'fileCreatedAt': file_created_at_timestamp,
            'fileModifiedAt': file_modified_at_timestamp
        }
        print(f"Uploading video to Immich with deviceAssetId: {video_filename}")
        response = requests.post(IMMICH_UPLOAD_URL, files=files, data=data, headers=headers)
        response.raise_for_status()
        print(f"Upload response code: {response.status_code}")
        print(response.json())
        if response.json()['status'] != 'created':
            print("Something went wrong!")
            raise RuntimeError()
        else:
            return True


In [ ]:
# === List all .mp4 files in the remote directory ===
result = conn.run(f"ls {REMOTE_ROOT}/*.mp4", hide=True, warn=True)
remote_files = result.stdout.strip().splitlines()
# print(remote_files)

if result.failed or not result.stdout.strip():
    print("No .mp4 files found in remote directory.")
else:
    remote_files = result.stdout.strip().splitlines()

    print(f"Found {len(remote_files)} video(s):")
    for remote_filepath in remote_files:
        video_name = os.path.basename(remote_filepath)
        local_filepath = os.path.join(TMP_DIR, video_name)

        print(f"Downloading {video_name}...")

        # Download file
        conn.get(remote_filepath, local_filepath)

        # Delete original
        conn.run(f"rm {remote_filepath}", hide=True)

        print(f"Moved {video_name}")
        print("Converting file")
        av1_filepath = convert_file(local_filepath)

        print("File converted, now uploading")
        success = send_image_to_immich(av1_filepath)
        if success == True:
            os.remove(local_filepath)
        else:
            raise RuntimeError()
        print("✅ Done!\n")
        # break
# 270ish videos is 31 mins
# 32 vids is 

Found 32 video(s):
Moved 20251023_222849_clip_04925_1.mp4
Converting file
File converted, now uploading
beginning to process the request
20251023_222849_clip_04925_1.mkv
2025-10-23T23:14:52.689061-06:00
Uploading video to Immich with deviceAssetId: 20251023_222849_clip_04925_1.mkv
Upload response code: 201
{'id': '8dfacd71-f9e1-4117-a129-b73a6121f721', 'status': 'created'}
✅ Done!

Moved 20251023_222959_clip_04928_1.mp4
Converting file
File converted, now uploading
beginning to process the request
20251023_222959_clip_04928_1.mkv
2025-10-23T23:15:04.307662-06:00
Uploading video to Immich with deviceAssetId: 20251023_222959_clip_04928_1.mkv
Upload response code: 201
{'id': '56f0099e-d0eb-4ee1-a1e6-fd558663d68f', 'status': 'created'}
✅ Done!

Moved 20251023_223109_clip_04931_1.mp4
Converting file
File converted, now uploading
beginning to process the request
20251023_223109_clip_04931_1.mkv
2025-10-23T23:15:15.626179-06:00
Uploading video to Immich with deviceAssetId: 20251023_223109_cli

CompletedProcess(args=['ffmpeg', '-hwaccel', 'cuda', '-i', './tmp/20251022_165042_clip_03709_1.mp4', '-c:v', 'av1_nvenc', '-preset', 'p7', '-cq', '37', '-c:a', 'copy', './tmp/20251022_165042_clip_03709_1.mkv'], returncode=0)

In [ ]:

        
send_image_to_immich(av1_filepath)

beginning to process the request
20251022_165042_clip_03709_1.mkv
2025-10-23T22:07:13.411836-06:00
Uploading video to Immich with deviceAssetId: 20251022_165042_clip_03709_1.mkv
Upload response code: 200
{'status': 'duplicate', 'id': '6ffb4546-95ff-4c7d-90c9-e828b5e208af'}
Something went wrong!
Error during send_image_to_immich
